In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl

In [ ]:
def discrete_laplacian(M):
    """Get the discrete Laplacian of matrix M"""
    L = -4*M
    L += np.roll(M, (0,-1), (0,1)) # right neighbor
    L += np.roll(M, (0,+1), (0,1)) # left neighbor
    L += np.roll(M, (-1,0), (0,1)) # top neighbor
    L += np.roll(M, (+1,0), (0,1)) # bottom neighbor
    
    return L

In [ ]:
def gray_scott_update(A, B, DA, DB, f, k, delta_t):
    """
    Updates a concentration configuration according to a Gray-Scott model
    with diffusion coefficients DA and DB, as well as feed rate f and
    kill rate k.
    """
    
    # Let's get the discrete Laplacians first
    LA = discrete_laplacian(A)
    LB = discrete_laplacian(B)
    
    # Now apply the update formula
    diff_A = (DA*LA - A*B**2 + f*(1-A)) * delta_t
    diff_B = (DB*LB + A*B**2 - (k+f)*B) * delta_t
    
    A += diff_A
    B += diff_B
    
    return A, B

In [ ]:

def get_initial_configuration(N, random_influence=0.2):
    """
    Initialize a concentration configuration. N is the side length
    of the (N x N)-sized grid.
    `random_influence` describes how much noise is added.
    """
    
    # We start with a configuration where on every grid cell 
    # there's a lot of chemical A, so the concentration is high
    A = (1-random_influence) * np.ones((N,N)) + random_influence * np.random.random((N,N))
    
    # Let's assume there's only a bit of B everywhere
    B = random_influence * np.random.random((N,N))
    
    # Now let's add a disturbance in the center
    N2 = N//2
    radius = r = int(N/10.0)
    
    A[N2-r:N2+r, N2-r:N2+r] = 0.50
    B[N2-r:N2+r, N2-r:N2+r] = 0.25
    
    return A, B

In [ ]:

def get_initial_configuration(N, random_influence=0.2):
    """
    Initialize a concentration configuration. N is the side length
    of the (N x N)-sized grid.
    `random_influence` describes how much noise is added.
    """
    
    # We start with a configuration where on every grid cell 
    # there's a lot of chemical A, so the concentration is high
    A = (1-random_influence) * np.ones((N,N)) + random_influence * np.random.random((N,N))
    
    # Let's assume there's only a bit of B everywhere
    B = random_influence * np.random.random((N,N))
    
    # Now let's add a disturbance in the center
    N2 = N//2
    radius = r = int(N/20.0)
    
    A[N2-r:N2+r, N2-r:N2+r] = 0.70
    B[N2-r:N2+r, N2-r:N2+r] = 0.40
    
    return A, B

In [ ]:
def draw(A,B):
    """draw the concentrations"""
    fig, ax = pl.subplots(1,2,figsize=(5.65,4))
    ax[0].imshow(A, cmap='Greys')
    ax[1].imshow(B, cmap='Greys')
    ax[0].set_title('A')
    ax[1].set_title('B')
    ax[0].axis('off')
    ax[1].axis('off')

In [ ]:
A, B = get_initial_configuration(200)
draw(A,B)

<IPython.core.display.Javascript object>

In [ ]:
# update in time
delta_t = 1.0

# coefficient values to get dividing droplets
# Diffusion coefficients
DA = 0.16
DB = 0.08

# define feed/kill rates
#f = 0.060
#k = 0.062
f=.0367
k=.0649

# grid size
N = 200

# simulation steps
N_simulation_steps = 10000

In [ ]:

A, B = get_initial_configuration(200)

for t in range(N_simulation_steps):
    A, B = gray_scott_update(A, B, DA, DB, f, k, delta_t)
    
draw(A,B)

<IPython.core.display.Javascript object>

In [ ]:
A, B = get_initial_configuration(200)

for t in range(N_simulation_steps):
    A, B = gray_scott_update(A, B, DA, DB, f, k, delta_t)
    
draw(A,B)

<IPython.core.display.Javascript object>

In [ ]:
import matplotlib.animation as animation  # this is needed for the animation API
from matplotlib.colors import Normalize   # this is needed to rescale the color during the simulation

In [ ]:
def get_artists(A,B):
    """draw the concentrations"""
    fig, ax = pl.subplots(1,2,figsize=(5.65,3))
    imA = ax[0].imshow(A, animated=True, cmap='Greys')
    imB = ax[1].imshow(B, animated=True, cmap='Greys')
    ax[0].set_title('A')
    ax[1].set_title('B')
    ax[0].axis('off')
    ax[1].axis('off')
    
    return fig, imA, imB

In [ ]:
def updatefig(frame_id,updates_per_frame,A,B,DA,DB,f,k,delta_t):
    """Takes care of the matplotlib-artist update in the animation"""
    global imA, imB

    # update x times before updating the frame
    for u in range(updates_per_frame):
        A, B = gray_scott_update(A,B,DA,DB,f,k,delta_t)

    # update the frame
    imA.set_array(A)
    imB.set_array(B)

    # renormalize the colors
    imA.set_norm(Normalize(vmin=np.amin(A),vmax=np.amax(A)))
    imB.set_norm(Normalize(vmin=np.amin(B),vmax=np.amax(B)))

    # return the updated matplotlib objects
    return imA, imB

In [ ]:
# Diffusion coefficients
DA = 0.16
DB = 0.08

# define feed/kill rates
f = 0.060
k = 0.062

N_simulation_steps = 20000

# intialize the figures
A, B = get_initial_configuration(N)
fig, imA, imB = get_artists(A, B)

# how many updates should be computed before a new frame is drawn
updates_per_frame = 10

# these are the arguments which have to passed to the update function
animation_arguments = (updates_per_frame, A, B, DA, DB, f, k, delta_t)

# let's make sure we can see the animation in the notebook
from IPython.display import HTML
mpl.rcParams['animation.html'] = 'html5'
#mpl.rcParams['animation.embed_limit'] = 1024 # this allows jupyter notebook to save 1024MB video data in the memory


# start the animation
ani = animation.FuncAnimation(fig, #matplotlib figure
                              updatefig, # function that takes care of the update
                              fargs=animation_arguments, # arguments to pass to this function
                              interval=10, # update every `interval` milliseconds
                              frames=int(N_simulation_steps/updates_per_frame), #limit the number of frames
                              blit=True, # optimize the drawing update
                              )

HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>

In [ ]:
DA = 0.16
DB = 0.08

# define feed/kill rates
#f = 0.060
#k = 0.062
f=.0367
k=.0649
#f=.03
#k=.06

N_simulation_steps = 10000

# intialize the figures
A, B = get_initial_configuration(N)
fig, imA, imB = get_artists(A, B)

# how many updates should be computed before a new frame is drawn
updates_per_frame = 10

# these are the arguments which have to passed to the update function
animation_arguments = (updates_per_frame, A, B, DA, DB, f, k, delta_t)

# let's make sure we can see the animation in the notebook
from IPython.display import HTML
mpl.rcParams['animation.html'] = 'html5'
#mpl.rcParams['animation.embed_limit'] = 1024 # this allows jupyter notebook to save 1024MB video data in the memory


# start the animation
ani = animation.FuncAnimation(fig, #matplotlib figure
                              updatefig, # function that takes care of the update
                              fargs=animation_arguments, # arguments to pass to this function
                              interval=10, # update every `interval` milliseconds
                              frames=int(N_simulation_steps/updates_per_frame), #limit the number of frames
                              blit=True, # optimize the drawing update
                              )

HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>